In [3]:
#!export PYTHONPATH=$PYTHONPATH:/data1/practical-sose23/castellvi/castellvi_prediction/bids
import os
import sys
sys.path.append('/data1/practical-sose23/castellvi/castellvi_prediction/bids')

In [4]:
from dataset.VerSe import VerSe
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, cohen_kappa_score, confusion_matrix
from modules.DenseNetModule import DenseNet
from utils._prepare_data import DataHandler
from utils.environment_settings import env_settings

import BIDS
from BIDS import BIDS_Global_info, BIDS_Family, NII
import nibabel as nib
import numpy as np
import cv2


import monai
#from monai.networks.utils import eval_mode
from monai.transforms import (
    AddChannel, Compose, LoadImage, RandRotate90, NormalizeIntensity, RandScaleIntensity,
    Resize, ScaleIntensityRange, ToTensor, ScaleIntensity, RandShiftIntensity,CropForeground,
    RandFlip, RandSpatialCrop, ResizeWithPadOrCrop, Spacing, ThresholdIntensity, Orientation
)
from monai.metrics import compute_roc_auc

import torch
from torch import nn

####################################
/u/home/ank/.conda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  File "/u/home/ank/.conda/envs/myenv/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/u/home/ank/.conda/envs/myenv/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/u/home/ank/.conda/envs/myenv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/u/home/ank/.conda/envs/myenv/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/u/home/ank/.conda/envs/myenv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/u/home/

2023-07-14 22:27:55,490 - Created a temporary directory at /tmp/tmpeu8t6xxu
2023-07-14 22:27:55,492 - Writing /tmp/tmpeu8t6xxu/_remote_module_non_scriptable.py


In [6]:
dataset = [os.path.join(env_settings.DATA, "dataset-verse19"), os.path.join(env_settings.DATA, "dataset-verse20"), os.path.join(env_settings.DATA, "dataset-tri")]
data_types = ['rawdata',"derivatives"]
image_types = ["ct", "subreg", "cortex"]

ckpt_path = "/data1/practical-sose23/castellvi/team_repo/3D-Castellvi-Prediction/experiments/baseline_models/densenet/best_models/version_0/densenet-epoch=38-val_mcc=0.91.ckpt"
master_list_new = '/data1/practical-sose23/castellvi/team_repo/3D-Castellvi-Prediction/src/dataset/Castellvi_list_Final_Split_v2.xlsx'
#Load model, processor and dataset
model = DenseNet.load_from_checkpoint(ckpt_path)
processor = DataHandler(master_list = master_list_new, dataset=dataset, data_types=data_types, image_types=image_types)
verse_dataset = VerSe(model.opt, processor, processor.verse_records + processor.tri_records, training=False)

DenseNet loaded with 3 classes, data_size (96, 78, 78) and 1 channel
[!] Unknown format seg-ano in file sub-verse649_dir-sag_seg-ano.nii.gz
[!] Unknown format iso-ctd in file sub-verse616_dir-iso_iso-ctd.json
[!] "template" is not a valid key/value pair. Expected "KEY-VALUE" in sub-verse519_template_sacrum_msk.nii.gz
[!] "sacrum" is not a valid key/value pair. Expected "KEY-VALUE" in sub-verse519_template_sacrum_msk.nii.gz
[!] Unknown format subreg in file verse559_CT-sag_seg-ano_subreg.nii.gz
[!] "verse559" is not a valid key/value pair. Expected "KEY-VALUE" in verse559_CT-sag_seg-ano_subreg.nii.gz
[!] "verse559" is not a valid key/value pair. Expected "KEY-VALUE" in verse559_CT-sag_seg-ano.nii.gz
[!] "verse549" is not a valid key/value pair. Expected "KEY-VALUE" in verse549_CT-iso_seg-ano.nii.gz
[!] Unknown format ce-ar in file sub-tri106_ce-ar.nii.gz
[!] Unknown format ce-pv in file sub-tri066_ce-pv.nii.gz
[!] Unknown format ce-ne in file sub-tri067_ce-ne.nii.gz
[!] Unknown format c

In [13]:
# For each record with dataset_split = val, get index of both flipped and non-flipped records
val_subjects = [record["subject"] for record in verse_dataset.records if (record["dataset_split"] == "val" and record["flip"] == 1)]
len(val_subjects)

val_subs_joined = {}
val_subs_idx = []
for subject in val_subjects:
    val_subs_joined[subject] = {}

In [14]:
# For each record with dataset_split = val, get index of both flipped and non-flipped records
val_records = [record for record in verse_dataset.records if (record["dataset_split"] == "val" and record["flip"] == 1)]
len(val_subjects)



73

In [16]:
from eval import Eval

In [18]:
evaluator = Eval(model.opt)

In [19]:
sample_record = val_records[0]
model_inputs = evaluator.process_input(processor, sample_record)
input_img = model_inputs['original'] # you can also get flipped one by model_inputs['flipped']
output_class, output_prob = evaluator.process_output(model, input_img)

AttributeError: 'Namespace' object has no attribute 'seg_comparison'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
campp = monai.visualize.GradCAMpp(nn_module = model, target_layers = "network.features.denseblock4.denselayer32.layers.conv2", )
win_size = (96, 78, 78)
print("original feature shape", campp.feature_map_size([1, 1] + list(win_size), device) )
print("upsampled feature shape", [1, 1] + list(win_size), device)

In [ ]:
def get_layer_names(model):
    layer_names = []
    for name, _ in model.named_parameters():
        # Split the name by '.' and keep the first part
        layer_name = name
        if layer_name not in layer_names:
            layer_names.append(layer_name)
    return layer_names

In [ ]:
layer_names = get_layer_names(model)

In [ ]:
conv_layers_list = []
for i in range(len(layer_names)):
    layer = layer_names[i]
    if 'conv' not in layer:
        continue
    else:
        conv_layers_list.append(layer)
    print(i , layer)

In [ ]:
campp_result = campp( x= input_img , class_idx=None)

In [ ]:
# Overlay Original image and GradCAM heatmap
def superimposed_image_def(camp_results, original_img, alpha = 0.6):
    all_images_slices = []
    all_heatmap_results = []
    print(input_img.shape[2]-1)
    for i in range(input_img.shape[-1]):
        print(i)
        original_image_slice= original_img.detach().cpu().numpy()[..., i]
        original_image_slice = original_image_slice[0,0,:,:] 
        print('original_img shape:', original_image_slice.shape)
        print('original_img data type:', original_image_slice.dtype)
        # Make 1-channel image to 3-channel : To overlay this image with the heatmap (JET colormap = 3-channels)
        original_image_slice = cv2.merge((original_image_slice, original_image_slice, original_image_slice))
        camp_result_slice = camp_results.detach().cpu().numpy()[..., i]
        camp_result_slice = camp_result_slice[0,0,:,:] # Same above
        print('cam_img shape:', camp_result_slice.shape)
        print('cam_img data type:', camp_result_slice.dtype)
        camp_result_slice = np.maximum(camp_result_slice, 0)


        # Convert Class Activation Map to 0 - 255
        camp_result_slice = (camp_result_slice - camp_result_slice.min()) / (
                camp_result_slice.max() - camp_result_slice.min()
        )
        camp_result_slice = np.uint8(255 * camp_result_slice)

        # Convert to Heatmap ---- JET COLORMAP
        camp_result_slice = cv2.applyColorMap(camp_result_slice, cv2.COLORMAP_JET)

        # Superimpose Heatmap on Image Data
        original_image_slice = np.uint8(
            (original_image_slice - original_image_slice.min())
            / (original_image_slice.max() - original_image_slice.min())
            * 255
        )

        superimposed_image =np.uint8(np.uint8(original_image_slice) * alpha + camp_result_slice * (1 - alpha))
        all_images_slices.append(superimposed_image)
        all_heatmap_results.append(camp_result_slice)
        

    return all_images_slices, all_heatmap_results

In [ ]:
imposed_img , heatmap = superimposed_image_def(campp_result, input_img)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(heatmap[30])
plt.axis('off')  
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(imposed_img[30])
plt.axis('off')  
plt.show()

In [ ]:
from matplotlib import animation, rc

In [ ]:
rc('animation', html='jshtml')


def create_animation(array):
    """Create an animation of a volume"""

    fig = plt.figure(figsize=(4, 4))
    images = []
    for idx, image in enumerate(array):
        # plot image without notifying animation
        image_plot = plt.imshow(image, animated=True, cmap='bone')
        aux = [image_plot]
        images.append(aux)

    plt.axis('off')
    plt.tight_layout()
    plt.subplots_adjust(top=0.90)
    plt.title(f'Deneme', fontsize=16)
    
    ani = animation.ArtistAnimation(
        fig, images, interval=5000//len(array), blit=False, repeat_delay=1000)
    plt.close()
    return ani

In [ ]:
create_animation(imposed_img)